In [198]:
from typing import Optional
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.stats import binned_statistic
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.ticker as tkr
from src.train.plots import Line
from src.processes.observables import Observable
import warnings

## Setup

In [199]:
plt.rc("font", family="serif", size=16)
plt.rc("axes", titlesize="medium")
plt.rc("text.latex", preamble=r"\usepackage{amsmath}")
plt.rc("text", usetex=True)

COLORS = [f"C{i}" for i in range(10)]
COLORS_exp = ["#b30000", "#7c1158", "#4421af", "#1a53ff", "#0d88e6", "#00b7c7", "#5ad45a", "#8be04e", "#ebdc78"]

RECO_COLOR = COLORS[0]
HARD_COLOR = COLORS[1]
HERWIG_RECO_COLOR = COLORS_exp[0]
HERWIG_HARD_COLOR = COLORS_exp[1]
AFF_INN_COLOR = COLORS[2]
INN_COLOR = "firebrick"
CFM_COLOR = COLORS_exp[3]
P_DIDI_COLOR = "forestgreen"
UP_DIDI_COLOR = "darkorchid"
SB_COLOR = COLORS[2]
OMNI_COLOR = COLORS[8]

mpl.rcParams['font.size'] = 29 # 17 for SB on second panel
                               # larger for migration like 22
mpl.rcParams['lines.linewidth'] = 5.
handlelength = .75 # handlelength in legend
handlewidth = 1.2 # handlelength in legend
observables_rect = (0.09,0.08,0.99,0.99) # to fix the axes box size in observbales plots

def hist_plot(
    pdf: PdfPages,
    lines: list[Line],
    bins: np.ndarray,
    observable: Observable,
    show_ratios: bool = True,
    title: Optional[str] = None,
    no_scale: bool = False,
    yscale: Optional[str] = None,
    show: bool = False,
    rect = None,
    ylabel = "Normalized",
    xlim = None,
    ylim = None,
    legend_kwargs = {},
    legend_callback = None,
    plot_SB_on_second_panel = False,
    debug: bool = False,
):
    """
    Makes a single histogram plot, used for the observable histograms and clustering
    histograms.
    Args:
        pdf: Multipage PDF object
        lines: List of line objects describing the histograms
        bins: Numpy array with the bin boundaries
        show_ratios: If True, show a panel with ratios
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", RuntimeWarning)

        n_panels = 1 + int(show_ratios) + int(plot_SB_on_second_panel)
        fig, axs = plt.subplots(
            n_panels,
            1,
            sharex=True,
            figsize=(8.5, 7),
            gridspec_kw={"height_ratios": (3, 1, 1)[:n_panels], "hspace": 0.00},
        )
        fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0.0, rect=rect)
        if n_panels == 1:
            axs = [axs]

        for line in lines:
            if line.vline:
                axs[0].axvline(line.y, label=line.label, color=line.color, linestyle=line.linestyle)
                continue
            integral = np.sum((bins[1:] - bins[:-1]) * line.y)
            scale = 1 / integral if integral != 0.0 else 1.0
            if line.y_ref is not None:
                ref_integral = np.sum((bins[1:] - bins[:-1]) * line.y_ref)
                ref_scale = 1 / ref_integral if ref_integral != 0.0 else 1.0
            if no_scale:
                scale = 1.
                ref_scale = 1.
            
            if debug: print("Actual values plotted:", line.y * scale)
            hist_line(
                axs[0],
                bins,
                line.y * scale,
                line.y_err * scale if line.y_err is not None else None,
                label=line.label,
                color=line.color,
                fill=line.fill,
                linestyle=line.linestyle
            )

            if line.y_ref is not None:
                ratio = (line.y * scale) / (line.y_ref * ref_scale)
                ratio_isnan = np.isnan(ratio) | np.isinf(ratio)
                if line.y_err is not None:
                    if len(line.y_err.shape) == 2:
                        ratio_err = (line.y_err * scale) / (line.y_ref * ref_scale)
                        ratio_err[:, ratio_isnan] = 0.0
                    else:
                        ratio_err = np.sqrt((line.y_err / line.y) ** 2)
                        ratio_err[ratio_isnan] = 0.0
                else:
                    ratio_err = None
                ratio[ratio_isnan] = 1.0
                if not plot_SB_on_second_panel or "SB" not in line.label:
                    hist_line(
                        axs[1], bins, ratio, ratio_err, label=None, color=line.color
                    )
                elif plot_SB_on_second_panel and "SB" not in line.label:
                    hist_line(
                        axs[1], bins, ratio, ratio_err, label=None, color=line.color
                    )
                else:
                    hist_line(
                        axs[2], bins, ratio, ratio_err, label=None, color=line.color
                    )
        if legend_callback is not None:
            legend_callback(axs)

        #if "Jet mass" in observable.tex_label:
        #    axs[0].legend(frameon=False, **legend_kwargs)
        axs[0].legend(frameon=False, **legend_kwargs)    

        axs[0].set_ylabel(ylabel)
        axs[0].set_yscale(observable.yscale if yscale is None else yscale)
        if ylim is not None:
            axs[0].set_ylim(*ylim)
        #if title is not None:
        #    self.corner_text(axs[0], title, "left", "top")
            
        if show_ratios:
            axs[1].set_ylabel(r"$\frac{\mathrm{Model}}{\mathrm{Truth}}$")
            axs[1].set_yticks([0.95, 1, 1.05])
            axs[1].set_ylim([0.9, 1.1])
            axs[1].axhline(y=1, c="black", ls="--", lw=0.7)
            axs[1].axhline(y=1.05, c="black", ls="dotted", lw=0.5)
            axs[1].axhline(y=0.95, c="black", ls="dotted", lw=0.5)
            if plot_SB_on_second_panel:
                axs[2].set_ylabel(r"$\frac{\mathrm{Model}}{\mathrm{Truth}}$")
                axs[2].set_yticks([0.95, 1, 1.05])
                axs[2].set_ylim([0.9, 1.1])
                axs[2].axhline(y=1, c="black", ls="--", lw=0.7)
                axs[2].axhline(y=1.05, c="black", ls="dotted", lw=0.5)
                axs[2].axhline(y=0.95, c="black", ls="dotted", lw=0.5)
        
        unit = "" if observable.unit is None else f" [{observable.unit}]"
        axs[-1].set_xlabel(f"${{{observable.tex_label}}}${unit}")
        axs[-1].set_xscale(observable.xscale)
        if xlim is None:
            axs[-1].set_xlim(bins[0], bins[-1])
        else:
            axs[-1].set_xlim(*xlim)

        plt.savefig(pdf, format="pdf")
        plt.close()

def hist_line(
    ax: mpl.axes.Axes,
    bins: np.ndarray,
    y: np.ndarray,
    y_err: np.ndarray,
    label: str,
    color: str,
    linestyle: str = "solid",
    fill: bool = False,
):
    """
    Plot a stepped line for a histogram, optionally with error bars.
    Args:
        ax: Matplotlib Axes
        bins: Numpy array with bin boundaries
        y: Y values for the bins
        y_err: Y errors for the bins
        label: Label of the line
        color: Color of the line
        linestyle: line style
        fill: Filled histogram
    """

    dup_last = lambda a: np.append(a, a[-1])

    if fill:
        ax.fill_between(
            bins, dup_last(y), label=label, facecolor=color, step="post", alpha=0.2
        )
    else:
        ax.step(
            bins,
            dup_last(y),
            label=label,
            color=color,
            linewidth=1.5,
            where="post",
            ls=linestyle,
        )
    if y_err is not None:
        if len(y_err.shape) == 2:
            y_low = y_err[0]
            y_high = y_err[1]
        else:
            y_low = y - y_err
            y_high = y + y_err

        ax.step(
            bins,
            dup_last(y_high),
            color=color,
            alpha=0.5,
            linewidth=0.5,
            where="post",
        )
        ax.step(
            bins,
            dup_last(y_low),
            color=color,
            alpha=0.5,
            linewidth=0.5,
            where="post",
        )
        ax.fill_between(
            bins,
            dup_last(y_low),
            dup_last(y_high),
            facecolor=color,
            alpha=0.3,
            step="post",
        )


## Pythia Small (BAYESIAN MODELS)

In [132]:
route_CFM = "output/CFM/20240221_104130_small-long-bay"
route_P_DIDI = "output/DiDi/paired/20240221_104350_small-long-bay"
route_UP_DIDI = "output/DiDi/unpaired/20240221_104446_small-long-bay"
route_INN = "output/INN/RQS/20240221_104809_small-long-bay"

routes = [route_CFM, route_P_DIDI, route_UP_DIDI, route_INN]
comparison_all = []
for route in routes:
    with open(route + "/observables.pkl", "rb") as f:
        comparison_all.append(pickle.load(f))

n_unfoldings = int(comparison_all[0][3]["lines"][3].label.split(" ")[0])
print(f"{n_unfoldings} UNFOLDINGS")
final_route = "plots/new_dataset/"

50 UNFOLDINGS


In [133]:
comparison_P_DIDI = comparison_all[1]
comparison_UP_DIDI = comparison_all[2]

with PdfPages(final_route + "SB_DiDi/observables-SB_DiDi-bay.pdf") as pp:
    for observable_P_DIDI, observable_UP_DIDI in zip(comparison_P_DIDI, comparison_UP_DIDI):

        
        # Reco
        line_reco = observable_P_DIDI["lines"][0]
        line_reco.color = RECO_COLOR
        line_reco.label = "Reco"
        
        # Hard
        line_hard = observable_P_DIDI["lines"][1]
        line_hard.color = HARD_COLOR
        line_hard.label = "Hard"

        # DiDi paired
        obs_P_DIDI = observable_P_DIDI["obs"]
        bins_P_DIDI = observable_P_DIDI["bins"]
        line_P_DIDI = observable_P_DIDI["lines"][3]
        line_P_DIDI.color = P_DIDI_COLOR
        line_P_DIDI.label = "DiDi-P"
        
        # DiDi unpaired
        obs_UP_DIDI = observable_UP_DIDI["obs"]
        bins_UP_DIDI = observable_UP_DIDI["bins"]
        line_UP_DIDI = observable_UP_DIDI["lines"][3]
        line_UP_DIDI.color = UP_DIDI_COLOR
        line_UP_DIDI.label = "DiDi-U"
        
        # SB
        #line_SB = observable_P_DIDI["lines"][4]
        #line_SB.color = "black"
        ylim = [.9, 1e1] if "momentum" in obs_P_DIDI.tex_label else None
        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_P_DIDI, line_UP_DIDI],#, line_SB],
            bins=bins_P_DIDI,
            observable=obs_P_DIDI,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs_P_DIDI.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs_P_DIDI.tex_label else [-0.01, 0.23] if "Groomed mass" in obs_P_DIDI.tex_label else [-0.1, 3.2] if "subjettiness" in obs_P_DIDI.tex_label else None,
            plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs_P_DIDI.tex_label or "multiplicity" in obs_P_DIDI.tex_label else "lower left" if "width" in obs_P_DIDI.tex_label else "upper left" if "Groomed mass" in obs_P_DIDI.tex_label or "subjettiness" in obs_P_DIDI.tex_label else "upper right" if "momentum" in obs_P_DIDI.tex_label else "best"), 'handlelength' : handlelength},
        )

In [134]:
comparison_CFM = comparison_all[0]
comparison_INN = comparison_all[3]

with PdfPages(final_route + "CFM_INN/observables-CFM_INN-bay.pdf") as pp:
    for observable_CFM, observable_INN in zip(comparison_CFM, comparison_INN):

        
        # Reco
        line_reco = observable_CFM["lines"][0]
        line_reco.color = RECO_COLOR
        line_reco.label = "Reco"
        
        # Hard
        line_hard = observable_CFM["lines"][1]
        line_hard.color = HARD_COLOR
        line_hard.label = "Hard"

        # CFM
        obs_CFM = observable_CFM["obs"]
        bins_CFM = observable_CFM["bins"]
        line_CFM = observable_CFM["lines"][3]
        line_CFM.color = CFM_COLOR
        line_CFM.label = "CFM"
        
        # INN
        obs_INN = observable_INN["obs"]
        bins_INN = observable_INN["bins"]
        line_INN = observable_INN["lines"][3]
        line_INN.color = INN_COLOR
        line_INN.label = "cINN"
        
        ylim = [.9, 1e1] if "momentum" in obs_INN.tex_label else None
        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_CFM, line_INN],#, line_SB],
            bins=bins_INN,
            observable=obs_INN,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs_INN.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs_INN.tex_label else [-0.01, 0.23] if "Groomed mass" in obs_INN.tex_label else [-0.1, 3.2] if "subjettiness" in obs_INN.tex_label else None,
            #plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs_INN.tex_label or "multiplicity" in obs_INN.tex_label else "lower left" if "width" in obs_INN.tex_label else "upper left" if "Groomed mass" in obs_INN.tex_label or "subjettiness" in obs_INN.tex_label else "upper right" if "momentum" in obs_INN.tex_label else "best"), 'handlelength' : handlelength},
        )

## Pythia Large (BAYESIAN MODELS)

In [200]:
route_CFM = "output/CFM/20240226_095540_large-long-bay"
route_P_DIDI = "output/DiDi/paired/20240226_095540_large-long-bay"
route_UP_DIDI = "output/DiDi/unpaired/20240226_095934_large-long-bay"
route_INN = "output/INN/RQS/20240226_111234_large-long-bay"

routes = [route_CFM, route_P_DIDI, route_UP_DIDI, route_INN]
comparison_all = []
for route in routes:
    with open(route + "/observables.pkl", "rb") as f:
        comparison_all.append(pickle.load(f))

n_unfoldings = int(comparison_all[0][3]["lines"][3].label.split(" ")[0])
print(f"{n_unfoldings} UNFOLDINGS")
final_route = "plots/new_dataset/"

50 UNFOLDINGS


In [201]:
comparison_P_DIDI = comparison_all[1]
comparison_UP_DIDI = comparison_all[2]

with PdfPages(final_route + "SB_DiDi/LARGE-observables-SB_DiDi-bay.pdf") as pp:
    for observable_P_DIDI, observable_UP_DIDI in zip(comparison_P_DIDI, comparison_UP_DIDI):

        
        # Reco
        line_reco = observable_P_DIDI["lines"][0]
        line_reco.color = RECO_COLOR
        line_reco.label = "Reco"
        
        # Hard
        line_hard = observable_P_DIDI["lines"][1]
        line_hard.color = HARD_COLOR
        line_hard.label = "Hard"

        # DiDi paired
        obs_P_DIDI = observable_P_DIDI["obs"]
        bins_P_DIDI = observable_P_DIDI["bins"]
        line_P_DIDI = observable_P_DIDI["lines"][3]
        line_P_DIDI.color = P_DIDI_COLOR
        line_P_DIDI.label = "DiDi-P"
        
        # DiDi unpaired
        obs_UP_DIDI = observable_UP_DIDI["obs"]
        bins_UP_DIDI = observable_UP_DIDI["bins"]
        line_UP_DIDI = observable_UP_DIDI["lines"][3]
        line_UP_DIDI.color = UP_DIDI_COLOR
        line_UP_DIDI.label = "DiDi-U"
        
        # SB
        #line_SB = observable_P_DIDI["lines"][4]
        #line_SB.color = "black"
        ylim = [.9, 1e1] if "momentum" in obs_P_DIDI.tex_label else None
        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_P_DIDI, line_UP_DIDI],#, line_SB],
            bins=bins_P_DIDI,
            observable=obs_P_DIDI,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs_P_DIDI.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs_P_DIDI.tex_label else [-0.01, 0.23] if "Groomed mass" in obs_P_DIDI.tex_label else [-0.1, 3.2] if "subjettiness" in obs_P_DIDI.tex_label else None,
            plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs_P_DIDI.tex_label or "multiplicity" in obs_P_DIDI.tex_label else "lower left" if "width" in obs_P_DIDI.tex_label else "upper left" if "Groomed mass" in obs_P_DIDI.tex_label or "subjettiness" in obs_P_DIDI.tex_label else "upper right" if "momentum" in obs_P_DIDI.tex_label else "best"), 'handlelength' : handlelength},
        )

In [202]:
comparison_CFM = comparison_all[0]
comparison_INN = comparison_all[3]

with PdfPages(final_route + "CFM_INN/LARGE-observables-CFM_INN-bay.pdf") as pp:
    for observable_CFM, observable_INN in zip(comparison_CFM, comparison_INN):

        
        # Reco
        line_reco = observable_CFM["lines"][0]
        line_reco.color = RECO_COLOR
        line_reco.label = "Reco"
        
        # Hard
        line_hard = observable_CFM["lines"][1]
        line_hard.color = HARD_COLOR
        line_hard.label = "Hard"

        # CFM
        obs_CFM = observable_CFM["obs"]
        bins_CFM = observable_CFM["bins"]
        line_CFM = observable_CFM["lines"][3]
        line_CFM.color = CFM_COLOR
        line_CFM.label = "CFM"
        
        # INN
        obs_INN = observable_INN["obs"]
        bins_INN = observable_INN["bins"]
        line_INN = observable_INN["lines"][3]
        line_INN.color = INN_COLOR
        line_INN.label = "cINN"
        
        ylim = [.9, 1e1] if "momentum" in obs_INN.tex_label else None
        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_CFM, line_INN],#, line_SB],
            bins=bins_INN,
            observable=obs_INN,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs_INN.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs_INN.tex_label else [-0.01, 0.23] if "Groomed mass" in obs_INN.tex_label else [-0.1, 3.2] if "subjettiness" in obs_INN.tex_label else None,
            #plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs_INN.tex_label or "multiplicity" in obs_INN.tex_label else "lower left" if "width" in obs_INN.tex_label else "upper left" if "Groomed mass" in obs_INN.tex_label or "subjettiness" in obs_INN.tex_label else "upper right" if "momentum" in obs_INN.tex_label else "best"), 'handlelength' : handlelength},
        )

## Migration2

### Small statistics

In [224]:
route_gt = "output/CFM/20240221_104130_small-long-bay/gt_"
route_SB = ""
route_CFM = "output/CFM/20240221_104130_small-long-bay/"
route_P_DIDI = "output/DiDi/paired/20240221_104350_small-long-bay/"
route_UP_DIDI = "output/DiDi/unpaired/20240221_104446_small-long-bay/"
route_INN = "output/INN/RQS/20240221_104809_small-long-bay/"

routes = [route_P_DIDI, route_UP_DIDI, route_CFM, route_INN, route_gt]#, route_SB]
migrations_all = []
for route in routes:
    with open(route + "migration2.pkl", "rb") as f:
        migrations_all.append(pickle.load(f))

final_route = "plots/new_dataset/migration/"

In [225]:
mpl.rcParams['font.size'] = 27.5 # 17 for SB on second panel
                               # larger for migration like 22
gt = False
reduce_titles = False
for i, prefix_fname, axis_label in zip(range(len(routes)), 
                                       ["DiDi_P", "DiDi_U", "CFM", "INN", "gt"],
                                       ["DiDi-P", "DiDi-U", "CFM", "cINN", "Hard"]):
    if i==4: gt = True
    #final_route = final_route + "title_only_on_gt/"
    with PdfPages(final_route + f"{prefix_fname}_migration2.pdf") as pp:
        for k, obs in enumerate(migrations_all[i]):
            h = obs["h"]
            bins = obs["bins"]
            ranges = obs["ranges"]
            obs = obs["obs"]
            '''if k == 0:
                obs.tex_label = "m"
            elif k == 1:
                obs.tex_label = "\omega"
            elif k == 2:
                obs.tex_label = "N"
            elif k == 3:
                obs.tex_label = "\log{\\rho}"
            elif k == 4:
                obs.tex_label = "z_{g}"
            elif k == 5:
                obs.tex_label = "\\tau_{21}"'''

            cmap = plt.get_cmap('viridis')
            cmap.set_bad("white")
            fig, ax = plt.subplots(figsize=(6,5))

            unit = "" if obs.unit is None else f" [{obs.unit}]"
            fig.suptitle(f"${{{obs.tex_label}}}${unit}")
            fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0.0, rect=(0.07, 0.08, 0.99, 0.99))

            
            im = ax.pcolormesh(bins, bins, h, cmap=cmap, rasterized=True, vmin=ranges[0], vmax=ranges[1])

            
            ax.set_xlim(bins[0], bins[-1])
            ax.set_ylim(bins[0], bins[-1])
            ax.set_xlabel("Reco")
            ax.set_ylabel(axis_label)
            fig.savefig(pp, format= "pdf")
            #fig.savefig(pp, format= "pdf", bbox_inches="tight")
            plt.close()

### Large statistics

In [226]:
route_gt = "output/CFM/20240226_095540_large-long-bay/gt_"
route_SB = ""
route_CFM = "output/CFM/20240221_104130_small-long-bay/"
route_P_DIDI = "output/DiDi/paired/20240226_095540_large-long-bay/"
route_UP_DIDI = "output/DiDi/unpaired/20240226_095934_large-long-bay/"
route_INN = "output/INN/RQS/20240226_111234_large-long-bay/"

routes = [route_P_DIDI, route_UP_DIDI, route_CFM, route_INN, route_gt]#, route_SB]
migrations_all = []
for route in routes:
    with open(route + "migration2.pkl", "rb") as f:
        migrations_all.append(pickle.load(f))

final_route = "plots/new_dataset/migration/LARGE-"

In [227]:
mpl.rcParams['font.size'] = 27.5 # 17 for SB on second panel
                               # larger for migration like 22
gt = False
reduce_titles = False
for i, prefix_fname, axis_label in zip(range(len(routes)), 
                                       ["DiDi_P", "DiDi_U", "CFM", "INN", "gt"],
                                       ["DiDi-P", "DiDi-U", "CFM", "cINN", "Hard"]):
    if i==4: gt = True
    #final_route = final_route + "title_only_on_gt/"
    with PdfPages(final_route + f"{prefix_fname}_migration2.pdf") as pp:
        for k, obs in enumerate(migrations_all[i]):
            h = obs["h"]
            bins = obs["bins"]
            ranges = obs["ranges"]
            obs = obs["obs"]
            '''if k == 0:
                obs.tex_label = "m"
            elif k == 1:
                obs.tex_label = "\omega"
            elif k == 2:
                obs.tex_label = "N"
            elif k == 3:
                obs.tex_label = "\log{\\rho}"
            elif k == 4:
                obs.tex_label = "z_{g}"
            elif k == 5:
                obs.tex_label = "\\tau_{21}"'''

            cmap = plt.get_cmap('viridis')
            cmap.set_bad("white")
            fig, ax = plt.subplots(figsize=(6,5))

            unit = "" if obs.unit is None else f" [{obs.unit}]"
            fig.suptitle(f"${{{obs.tex_label}}}${unit}")
            fig.tight_layout(pad=0.0, w_pad=0.0, h_pad=0.0, rect=(0.07, 0.08, 0.99, 0.99))

            
            im = ax.pcolormesh(bins, bins, h, cmap=cmap, rasterized=True, vmin=ranges[0], vmax=ranges[1])

            
            ax.set_xlim(bins[0], bins[-1])
            ax.set_ylim(bins[0], bins[-1])
            ax.set_xlabel("Reco")
            ax.set_ylabel(axis_label)
            fig.savefig(pp, format= "pdf")
            #fig.savefig(pp, format= "pdf", bbox_inches="tight")
            plt.close()

## Omnifold vs bOmnifold

### Reweighting Herwig onto Pythia

In [184]:
route_omnifold = "output/Omnifold/20240226_155542_olddatasets-HtoP-long"
route_bOmnifold = "output/Omnifold/20240226_155816_olddatasets-HtoP-long-bay"

routes = [route_omnifold, route_bOmnifold]

reco_train_all = []
for route in routes:
    with open(route + "/reco_train.pkl", "rb") as f:
        reco_train_all.append(pickle.load(f))

reco_all = []
for route in routes:
    with open(route + "/reco.pkl", "rb") as f:
        reco_all.append(pickle.load(f))

observables_all = []
for route in routes:
    with open(route + "/observables.pkl", "rb") as f:
        observables_all.append(pickle.load(f))

final_route = "plots/Omnifold/HtoP/"

In [185]:
'''
TRAINING SET RECO-LEVEL DISTRIBUTIONS
'''
with PdfPages(final_route + "HtoP-reco_train.pdf") as pp:
    for reco_train_omnifold, reco_train_bOmnifold in zip(reco_train_all[0], reco_train_all[1]):
        
        obs = reco_train_bOmnifold["obs"]
        bins = reco_train_bOmnifold["bins"]

        # Pythia Reco
        line_Pythia_reco = reco_train_bOmnifold["lines"][0]
        line_Pythia_reco.color = HARD_COLOR
        #line_Pythia_reco.label = "Pythia Reco"

        # Herwig Reco
        line_Herwig_reco = reco_train_bOmnifold["lines"][1]
        line_Herwig_reco.color = RECO_COLOR
        #line_Herwig_reco.label = "Herwig Reco"

        # omnifold reweighted
        line_omnifold = reco_train_omnifold["lines"][2]
        line_omnifold.color = AFF_INN_COLOR

        # bOmnifold reweighted
        line_bOmnifold = reco_train_bOmnifold["lines"][2]
        line_bOmnifold.color = "firebrick"
        
        hist_plot(
            pdf=pp,
            lines=[line_Pythia_reco, line_Herwig_reco, line_omnifold, line_bOmnifold],
            bins=bins,
            observable=obs,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )



'''
RECO-LEVEL DISTRIBUTIONS
'''
with PdfPages(final_route + "HtoP-reco.pdf") as pp:
    for reco_omnifold, reco_bOmnifold in zip(reco_all[0], reco_all[1]):
        
        obs = reco_bOmnifold["obs"]
        bins = reco_bOmnifold["bins"]

        # Pythia Reco
        line_Pythia_reco = reco_bOmnifold["lines"][0]
        line_Pythia_reco.color = HARD_COLOR
        #line_Pythia_reco.label = "Pythia Reco"

        # Herwig Reco
        line_Herwig_reco = reco_bOmnifold["lines"][1]
        line_Herwig_reco.color = RECO_COLOR
        #line_Herwig_reco.label = "Herwig Reco"

        # omnifold reweighted
        line_omnifold = reco_omnifold["lines"][2]
        line_omnifold.color = AFF_INN_COLOR
        line_omnifold.label = "Omnifold"

        # bOmnifold reweighted
        line_bOmnifold = reco_bOmnifold["lines"][2]
        line_bOmnifold.color = "firebrick"
        line_bOmnifold.label = "bOmnifold"
        
        hist_plot(
            pdf=pp,
            lines=[line_Pythia_reco, line_Herwig_reco, line_omnifold, line_bOmnifold],
            bins=bins,
            observable=obs,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )



'''
OBSERVABLES
'''
with PdfPages(final_route + "HtoP-observables.pdf") as pp:
    for observables_omnifold, observables_bOmnifold in zip(observables_all[0], observables_all[1]):
        
        obs = observables_bOmnifold["obs"]
        bins = observables_bOmnifold["bins"]

        # Pythia Reco
        line_reco = observables_bOmnifold["lines"][0]
        line_reco.color = RECO_COLOR
        #line_reco.label = "Herwig Reco"

        # Herwig Hard 
        line_hard = observables_bOmnifold["lines"][1]
        line_hard.color = HARD_COLOR
        #line_hard.label = "Herwig Hard"
        
        # omnifold reweighted
        line_omnifold = observables_omnifold["lines"][2]
        line_omnifold.color = AFF_INN_COLOR
        line_omnifold.label = "Omnifold"
        
        # bOmnifold reweighted
        line_bOmnifold = observables_bOmnifold["lines"][2]
        line_bOmnifold.color = "firebrick"
        line_bOmnifold.label = "bOmnifold"
        
        ylim = [5e-1, 1e1] if "momentum" in obs.tex_label else None

        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_omnifold, line_bOmnifold],
            bins=bins,
            observable=obs,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )

### Noisy pythia only

In [187]:
route_omnifold = "output/Omnifold/20240226_161313_onlypythia-long"
route_bOmnifold = "output/Omnifold/20240226_161539_onlypythia-long-bay"

routes = [route_omnifold, route_bOmnifold]

reco_train_all = []
for route in routes:
    with open(route + "/reco_train.pkl", "rb") as f:
        reco_train_all.append(pickle.load(f))

reco_all = []
for route in routes:
    with open(route + "/reco.pkl", "rb") as f:
        reco_all.append(pickle.load(f))

observables_all = []
for route in routes:
    with open(route + "/observables.pkl", "rb") as f:
        observables_all.append(pickle.load(f))

final_route = "plots/Omnifold/PtoP/"

In [188]:
'''
TRAINING SET RECO-LEVEL DISTRIBUTIONS
'''
with PdfPages(final_route + "PtoP-reco_train.pdf") as pp:
    for reco_train_omnifold, reco_train_bOmnifold in zip(reco_train_all[0], reco_train_all[1]):
        
        obs = reco_train_bOmnifold["obs"]
        bins = reco_train_bOmnifold["bins"]

        # Pythia Reco 1
        line_Pythia_reco1 = reco_train_bOmnifold["lines"][0]
        line_Pythia_reco1.color = RECO_COLOR
        line_Pythia_reco1.y_ref = None

        # Pythia Reco 2
        line_Pythia_reco2 = reco_train_bOmnifold["lines"][1]
        line_Pythia_reco2.color = HARD_COLOR


        # omnifold reweighted
        line_omnifold = reco_train_omnifold["lines"][2]
        line_omnifold.color = AFF_INN_COLOR

        # bOmnifold reweighted
        line_bOmnifold = reco_train_bOmnifold["lines"][2]
        line_bOmnifold.color = "firebrick"
        
        hist_plot(
            pdf=pp,
            lines=[line_Pythia_reco1, line_Pythia_reco2, line_omnifold, line_bOmnifold],
            bins=bins,
            observable=obs,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )



'''
RECO-LEVEL DISTRIBUTIONS
'''
with PdfPages(final_route + "PtoP-reco.pdf") as pp:
    for reco_omnifold, reco_bOmnifold in zip(reco_all[0], reco_all[1]):
        
        obs = reco_bOmnifold["obs"]
        bins = reco_bOmnifold["bins"]

        # Pythia Reco 1
        line_Pythia_reco1 = reco_bOmnifold["lines"][0]
        line_Pythia_reco1.color = RECO_COLOR
        line_Pythia_reco1.y_ref = None

        # Pythia Reco 2
        line_Pythia_reco2 = reco_bOmnifold["lines"][1]
        line_Pythia_reco2.color = HARD_COLOR


        # omnifold reweighted
        line_omnifold = reco_omnifold["lines"][2]
        line_omnifold.color = AFF_INN_COLOR

        # bOmnifold reweighted
        line_bOmnifold = reco_bOmnifold["lines"][2]
        line_bOmnifold.color = "firebrick"
        
        hist_plot(
            pdf=pp,
            lines=[line_Pythia_reco1, line_Pythia_reco2, line_omnifold, line_bOmnifold],
            bins=bins,
            observable=obs,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )



'''
OBSERVABLES
'''
with PdfPages(final_route + "PtoP-observables.pdf") as pp:
    for observables_omnifold, observables_bOmnifold in zip(observables_all[0], observables_all[1]):
        
        obs = observables_bOmnifold["obs"]
        bins = observables_bOmnifold["bins"]

        # Pythia Reco 2
        line_Pythia_reco2 = observables_bOmnifold["lines"][0]
        line_Pythia_reco2.color = RECO_COLOR

        # Pythia Hard 2
        line_Pythia_hard2 = observables_bOmnifold["lines"][1]
        line_Pythia_hard2.color = HARD_COLOR
        
        # omnifold reweighted
        line_omnifold = observables_omnifold["lines"][2]
        line_omnifold.color = AFF_INN_COLOR
        
        # bOmnifold reweighted
        line_bOmnifold = observables_bOmnifold["lines"][2]
        line_bOmnifold.color = "firebrick"
        
        ylim = [1e0, 1e1] if "momentum" in obs.tex_label else None

        hist_plot(
            pdf=pp,
            lines=[line_Pythia_reco2, line_Pythia_hard2, line_omnifold, line_bOmnifold],
            bins=bins,
            observable=obs,
            show_ratios=True,
            xlim = [0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[.9, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )

## Plot dataset

In [138]:
route = "output/DiDi/paired/20240221_104350_small-long-bay"
with open(route + "/observables.pkl", "rb") as f:
    Pythia = pickle.load(f)
final_route = "plots/new_dataset/"

In [139]:
Pythia_hard = []
Pythia_reco = []
for k in range(len(Pythia)):
    Pythia_reco.append(Pythia[k]["lines"][0])
    Pythia_hard.append(Pythia[k]["lines"][1])

In [140]:
with PdfPages(final_route + "new_dataset.pdf") as pp:
    for observables_Pythia in Pythia:
        
        obs = observables_Pythia["obs"]
        bins = observables_Pythia["bins"]

        # Pythia Reco
        line_Pythia_reco = observables_Pythia["lines"][0]
        line_Pythia_reco.color = RECO_COLOR
        line_Pythia_reco.label = "Reco"
        line_Pythia_reco.linestyle = "solid"
        
        # Pythia Hard
        line_Pythia_hard = observables_Pythia["lines"][1]
        line_Pythia_hard.color = HARD_COLOR
        line_Pythia_hard.label = "Hard"
        line_Pythia_hard.linestyle = "solid"

        hist_plot(
            pdf=pp,
            lines=[line_Pythia_reco, line_Pythia_hard],
            bins=bins,
            observable=obs,
            show_ratios=False,
            xlim = [-0.05, 0.55] if "momentum" in obs.tex_label else None,
            ylim=[5e-1, 1e1] if "momentum" in obs.tex_label else [-0.01, 0.23] if "Groomed mass" in obs.tex_label else [-0.1, 3.2] if "subjettiness" in obs.tex_label else None,
            #plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs.tex_label or "multiplicity" in obs.tex_label else "lower left" if "width" in obs.tex_label else "upper left" if "Groomed mass" in obs.tex_label or "subjettiness" in obs.tex_label else "upper right" if "momentum" in obs.tex_label else "best"), 'handlelength' : handlelength},
        )

## Effects of preprocessing

In [161]:
route_INN_nopreproc = "output/INN/RQS/20240226_085520_nopreproc-bay"
route_P_DIDI_nopreproc = "output/DiDi/paired/20240227_154514_nopreproc-small-short-bay-big_batch"
route_INN_both_preproc = "output/INN/RQS/20240220_181451_small-short-bay"
route_P_DIDI_both_preproc = "output/DiDi/paired/20240227_130018_small-short-bay-big_batch"

routes = [route_INN_nopreproc, route_P_DIDI_nopreproc, route_INN_both_preproc, route_P_DIDI_both_preproc]
comparison_all = []
for route in routes:
    with open(route + "/observables.pkl", "rb") as f:
        comparison_all.append(pickle.load(f))

n_unfoldings = int(comparison_all[0][3]["lines"][3].label.split(" ")[0])
print(f"{n_unfoldings} UNFOLDINGS")
final_route = "plots/new_dataset/"

50 UNFOLDINGS


In [162]:
INN_nopreproc = comparison_all[0]
P_DIDI_nopreproc = comparison_all[1]

with PdfPages(final_route + "preprocessing/no_preproc.pdf") as pp:
    for observable_INN, observable_P_DIDI in zip(INN_nopreproc, P_DIDI_nopreproc):

        # Reco
        line_reco = observable_INN["lines"][0]
        line_reco.color = RECO_COLOR
        line_reco.label = "Reco"
        
        # Hard
        line_hard = observable_INN["lines"][1]
        line_hard.color = HARD_COLOR
        line_hard.label = "Hard"

        # INN
        obs_INN = observable_INN["obs"]
        bins_INN = observable_INN["bins"]
        line_INN = observable_INN["lines"][3]
        line_INN.color = INN_COLOR
        line_INN.label = "cINN/CFM"
        
        # P_DIDI
        obs_P_DIDI = observable_P_DIDI["obs"]
        bins_P_DIDI = observable_P_DIDI["bins"]
        line_P_DIDI = observable_P_DIDI["lines"][3]
        line_P_DIDI.color = P_DIDI_COLOR
        line_P_DIDI.label = "DiDi/SB"

        
        # SB
        #line_SB = observable_P_DIDI["lines"][4]
        #line_SB.color = "black"
        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_INN, line_P_DIDI],#, line_SB],
            bins=bins_P_DIDI,
            observable=obs_P_DIDI,
            show_ratios=True,
            xlim = None,
            ylim=[5e-3, 1e1] if "momentum" in obs_P_DIDI.tex_label else [-0.005, 0.255] if "Groomed mass" in obs_P_DIDI.tex_label else [-0.1, 3.2] if "subjettiness" in obs_P_DIDI.tex_label else None,
            #plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs_P_DIDI.tex_label or "multiplicity" in obs_P_DIDI.tex_label else "lower left" if "width" in obs_P_DIDI.tex_label else "upper left" if "Groomed mass" in obs_P_DIDI.tex_label or "subjettiness" in obs_P_DIDI.tex_label else "lower center" if "momentum" in obs_P_DIDI.tex_label else "best"), 'handlelength' : handlelength},
        )

In [163]:
INN_both_preproc = comparison_all[2]
P_DIDI_both_preproc = comparison_all[3]

with PdfPages(final_route + "preprocessing/both_preproc.pdf") as pp:
    for observable_INN, observable_P_DIDI in zip(INN_both_preproc, P_DIDI_both_preproc):

        # Reco
        line_reco = observable_INN["lines"][0]
        line_reco.color = RECO_COLOR
        line_reco.label = "Reco"
        
        # Hard
        line_hard = observable_INN["lines"][1]
        line_hard.color = HARD_COLOR
        line_hard.label = "Hard"

        # INN
        obs_INN = observable_INN["obs"]
        bins_INN = observable_INN["bins"]
        line_INN = observable_INN["lines"][3]
        line_INN.color = INN_COLOR
        line_INN.label = "cINN/CFM"
        
        # P_DIDI
        obs_P_DIDI = observable_P_DIDI["obs"]
        bins_P_DIDI = observable_P_DIDI["bins"]
        line_P_DIDI = observable_P_DIDI["lines"][3]
        line_P_DIDI.color = P_DIDI_COLOR
        line_P_DIDI.label = "DiDi/SB"
        
        # SB
        #line_SB = observable_P_DIDI["lines"][4]
        #line_SB.color = "black"
        hist_plot(
            pdf=pp,
            lines=[line_reco, line_hard, line_INN, line_P_DIDI],#, line_SB],
            bins=bins_P_DIDI,
            observable=obs_P_DIDI,
            show_ratios=True,
            xlim = None,
            ylim=[5e-3, 1e1] if "momentum" in obs_P_DIDI.tex_label else [-0.005, 0.255] if "Groomed mass" in obs_P_DIDI.tex_label else [-0.1, 3.2] if "subjettiness" in obs_P_DIDI.tex_label else None,
            #plot_SB_on_second_panel=True,
            show=True,
            rect=observables_rect, # left, bottom, right, top
            legend_kwargs={"loc": ("upper right" if "Jet mass" in obs_P_DIDI.tex_label or "multiplicity" in obs_P_DIDI.tex_label else "lower left" if "width" in obs_P_DIDI.tex_label else "upper left" if "Groomed mass" in obs_P_DIDI.tex_label or "subjettiness" in obs_P_DIDI.tex_label else "lower center" if "momentum" in obs_P_DIDI.tex_label else "best"), 'handlelength' : handlelength},
        )